In [11]:
import pathlib as pt
import scipy.io as scio
import matplotlib.pyplot as plt
import torch

We can use the [pathlib]("https://docs.python.org/3/library/pathlib.html") library to read the files within a folder

In [16]:
current_folder = pt.Path(".")
mat_folder = current_folder/"mat_files"
first_mat_file = next(iter(mat_folder.glob("*.mat")))

The scipy.io function "[loadmat](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html)" will load the dictionary containing our mat data

In [20]:
data_dict = scio.loadmat(first_mat_file)
data_dict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'h', 'th', 'wss_z', 'z'])

We need to create a new dictionary which defines the inputs "x" and outputs "y" of our deep learning model. The inputs of the dictionary will be tensors of dimensions channels, height and width. First we will convert our data into torch tensors. We can do this for our data in one line using a [dictioanry comprehension](https://www.programiz.com/python-programming/dictionary-comprehension).

In [30]:
torch_tensor_data = {key: torch.from_numpy(data) for key, data in data_dict.items() if not key.startswith("_")}

Now that our data has been converted into torch tensor we can create the "x" and "y" tensors

In [26]:
x = torch.stack(
    [
        data_dict["h"], data_dict["th"], data_dict["z"]
    ]
)

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray